In [20]:
%pwd

'/home/codespace/FL-DP'

In [26]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
from textblob import Word
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet

import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
def cleaning_smsspam_dataset(df_data):
    # covert uppercase letters to lowercase letters
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        x.lower() for x in x.split()
    ))

    # delete puctuation marks
    df_data['text'] = df_data['text'].str.replace('[^\w\s]', '')

    # delete numbers from texts
    df_data['text'] = df_data['text'].str.replace('\d', '')

    # delete stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = list(string.punctuation)
    stop_words.update(punctuation)
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        x for x in x.split() if x not in stop_words
    ))

    # lemmatization and get the roots of the words
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        [Word(word).lemmatize() for word in x.split()]
    ))

    # remove words less than 3 letters
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        [x for x in x.split() if len(x) > 3]
    ))
    
    return df_data


def preprocessing_smsspam_dataset(datafile):
    # load dataset
    data = pd.read_csv(datafile, encoding='ISO-8859-1', 
                   engine='python')
    #rename dataset columns
    data.rename(columns = {"v1": "target", "v2": "text"}, inplace = True)

    #drop unnecessary columns
    data.drop(["Unnamed: 2","Unnamed: 3", "Unnamed: 4"], axis = 1, inplace = True)

    # drop duplicate data
    data.drop_duplicates(inplace = True)

    # cleaning data
    data = cleaning_smsspam_dataset(data)

    return data

In [35]:
def smsspam_load_data(test_prob=0.2, max_len=71):
    data_file = './datasets/smsspam/spam.csv'
    data = preprocessing_smsspam_dataset(data_file)

    x_data = data['text'].values
    y_data = data['target'].values

    print(x_data[0])
    print(x_data[1])
    tokenizer = get_tokenizer(tokenizer='spacy', language='en_core_web_sm')
    
    # print(x_data)
    vocab = build_vocab_from_iterator([tokenizer(text) for text in x_data])

    text_pipeline = lambda x: vocab(tokenizer(x))
    
    x_data = [text_pipeline(text) for text in x_data]
    x_data = [np.asarray(sample, dtype=np.int32) for sample in x_data]
    x_data = [np.pad(sample, (0, max(0, max_len-len(sample))), mode='constant', constant_values=0) for sample in x_data]
    print(type(x_data))
    print(x_data[0].shape)
    print(x_data[1])

In [36]:
smsspam_load_data()

/tmp/ipykernel_1596/1833105525.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df_data['text'] = df_data['text'].str.replace('[^\w\s]', '')
/tmp/ipykernel_1596/1833105525.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_data['text'] = df_data['text'].str.replace('\d', '')


jurong point crazy available bugis great world buffet cine amore
joking
<class 'list'>
(71,)
[897   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [37]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   testing.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
